# Scraping af FT

In [126]:
from bs4 import BeautifulSoup as bs
import re 
import os
import lxml
import xml.etree.ElementTree as ET
from lxml import objectify
from datetime import datetime
import numpy as np 
import pandas as pd 
import requests
import ftplib
from io import BytesIO
import tqdm

### Prepare to load data

In [127]:
ftp_url = "oda.ft.dk"
ftp_dir = "/ODAXML/Referat/samling/"


##### Tomme lister

In [128]:
dirs = []
sub_dir = []
files = []
file_name = []
paths = []
raw_xml_list = []

suppe = []

### Login til ftp serveren og henter en liste over biblioteker

In [129]:
ftp = ftplib.FTP(ftp_url)
ftp.login("anonymous", "wpg345@alumni.ku.dk")
ftp.set_pasv(True)
ftp.cwd(ftp_dir)
ftp.dir(dirs.append)

### Bygger en liste med underbibliotekter

In [130]:
for i in range (0, len(dirs), 1):
    sub_dir.append(ftp_dir+dirs[i][-5:]+'/') 

### Bygger en liste med filnavne

In [131]:
for i in range (0, len(sub_dir), 1):
    ftp.cwd(sub_dir[i])
    ftp.dir(files.append)

### Ekstrakt af filnavn

In [132]:
for i in range(0, len(files)):
    file_name.append(files[i][39:])

### Opbygning af hele stien og filnavn

In [133]:
for i in range(0, len(file_name)):
    paths.append(ftp_dir+file_name[i][0:5]+'/'+file_name[i])

### Overførsel af de enkelte filer fra ftp://oda.ft.dk, og raw_xml skrives til liste

In [134]:
for i in tqdm.tqdm(range(0, len(paths))):
    r = BytesIO()
    ftp.retrbinary(f"RETR {paths[i]}", r.write)
    raw_xml_list.append(r.getvalue())
    r.close()

100%|██████████| 1489/1489 [03:15<00:00,  7.63it/s]


# Define metadata function

In [135]:
def extract_metadata(xml): 
    soup = bs(xml, 'lxml')
    
    d = soup.find("dateofsitting").text[:10]
    m = soup.find("meetingnumber").text
    s = soup.find("parliamentarysession").text
    g = soup.find("parliamentarygroup").text
    l = soup.find("location").text
    
    
    repeats = len(soup.find_all("dagsordenpunkt"))
    
    for repeat in range(0, repeats):
        date.append(d)
        meetingnumber.append(m)
        session.append(s)
        group.append(g)
        location.append(l)

In [139]:
# empty lists (columns)
date = []
content = []
meetingnumber = []
session = []
group = []
location = []

# Define content function 

In [140]:
def extract_content(xml):
    soup = bs(xml, 'lxml')
    
    dp = soup.find_all("dagsordenpunkt")
    
    for punkt in range(0, len(dp)):
        content.append(dp[punkt].get_text(separator = ' '))

# Sample run

In [137]:
'''
xml_sample = raw_xml_list[0:5]

for xml in xml_sample: 
    extract_metadata(xml)

for xml in xml_sample: 
    extract_content(xml)
'''

# Full run

In [143]:
for xml in tqdm.tqdm(raw_xml_list): 
    extract_metadata(xml)
    extract_content(xml)

100%|██████████| 1489/1489 [16:27<00:00,  1.51it/s]


# Make DataFrame

In [165]:
df = pd.DataFrame([date, content, meetingnumber, session, group, location]).transpose()
df.columns = ['date', 'content', 'meetingnumber', 'session', 'group', 'location']

In [171]:
df

,date,content,meetingnumber,session,group,location
0,2010-05-31,0 FM Punkt 0 Meddelelser fra formanden Holger ...,100,20091,Folketinget,Folketingssalen
1,2010-05-31,1 222 L BEH1 1. behandling af L 222: Om nedsæt...,100,20091,Folketinget,Folketingssalen
2,2010-05-31,2 218 L BEH1 1. behandling af L 218: Om nedsæt...,100,20091,Folketinget,Folketingssalen
3,2010-05-31,3 219 L BEH1 1. behandling af L 219: Om forhøj...,100,20091,Folketinget,Folketingssalen
4,2010-05-31,4 223 L BEH1 1. behandling af L 223: Om staten...,100,20091,Folketinget,Folketingssalen
...,...,...,...,...,...,...
16762,2021-10-26,None,9,20211,Folketinget,Folketingssalen
16763,2021-10-26,None,9,20211,Folketinget,Folketingssalen
16764,2021-10-26,None,9,20211,Folketinget,Folketingssalen
16765,2021-10-26,None,9,20211,Folketinget,Folketingssalen


# Examining the empty content rows 

De sidste 78 rækker har ingen content. Vi vil undersøge det nærmere i cleaning-delen. 

In [170]:
df[df['content'].notnull()]

,date,content,meetingnumber,session,group,location
0,2010-05-31,0 FM Punkt 0 Meddelelser fra formanden Holger ...,100,20091,Folketinget,Folketingssalen
1,2010-05-31,1 222 L BEH1 1. behandling af L 222: Om nedsæt...,100,20091,Folketinget,Folketingssalen
2,2010-05-31,2 218 L BEH1 1. behandling af L 218: Om nedsæt...,100,20091,Folketinget,Folketingssalen
3,2010-05-31,3 219 L BEH1 1. behandling af L 219: Om forhøj...,100,20091,Folketinget,Folketingssalen
4,2010-05-31,4 223 L BEH1 1. behandling af L 223: Om staten...,100,20091,Folketinget,Folketingssalen
...,...,...,...,...,...,...
16684,2022-04-21,4 16 L BEH1 1. behandling af L 16: Om ændring ...,93,20211,Folketinget,Folketingssalen
16685,2022-04-21,5 27 L BEH1 1. behandling af L 27: Om forenkli...,94,20211,Folketinget,Folketingssalen
16686,2022-04-21,6 10 L BEH1 1. behandling af L 10: Om ophævels...,94,20211,Folketinget,Folketingssalen
16687,2022-04-21,7 11 L BEH1 1. behandling af L 11: Om tilpasni...,94,20211,Folketinget,Folketingssalen


Metadata om de berørte rækker.

In [176]:
df.iloc[16689:].date.unique()

array(['2022-04-21', '2022-04-22', '2022-04-25', '2022-04-26',
       '2022-04-27', '2022-04-28', '2021-10-26'], dtype=object)

In [177]:
df.iloc[16689:].meetingnumber.unique()

array(['94', '95', '96', '97', '98', '99', '9'], dtype=object)

In [178]:
df.iloc[16689:].session.unique()

array(['20211'], dtype=object)

# Save df to csv

In [164]:
df.to_csv('FT.csv', index = False)